In [1]:
!pip install tensorflow-text

^C
ERROR: Operation cancelled by user


In [2]:
import pandas as pd
import nltk
import gensim
nltk.download('popular')

KeyboardInterrupt: 

In [ ]:
!wget -c https://firebasestorage.googleapis.com/v0/b/william-personal-firebase.appspot.com/o/chatbot.zip?alt=media&token=f30ea369-1ffc-4163-baef-45b2e2f883c6 
!unzip /content/chatbot.zip?alt=media

In [ ]:
df = pd.read_csv('content/archive/20000-Utterances-Training-dataset-for-chatbots-virtual-assistant-Bitext-sample/20000-Utterances-Training-dataset-for-chatbots-virtual-assistant-Bitext-sample/20000-Utterances-Training-dataset-for-chatbots-virtual-assistant-Bitext-sample.csv')
df

Basic processing

In [ ]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

porter = PorterStemmer()
english_stopwords = stopwords.words('english')

# lower case, tokenize, stopword, puctuation and stemming
df['words'] = df['utterance'].str.lower()
df['words'] = df['words'].apply(word_tokenize)
df['words'] = df['words'].apply(lambda l: [porter.stem(word) for word in l if (word not in english_stopwords) and word.isalpha()])
print(df['words'].head())
print(df['words'].describe())

In [ ]:
from nltk.probability import FreqDist

fdist = FreqDist(df['words'].sum())
fdist.most_common(3)
fdist.plot(10)


Train test Split

In [ ]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2, random_state = 1337)

Count vectorize

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# merged words list
train_words=[' '.join(text) for text in df_train['words']]
test_words=[' '.join(text) for text in df_test['words']]

# count
count_vectorizer = CountVectorizer()
X_train_counts = count_vectorizer.fit_transform(train_words).toarray()
X_test_counts = count_vectorizer.transform(test_words).toarray()

Ngram vectorize

In [ ]:
# ngram
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
X_train_ng = ngram_vectorizer.fit_transform(train_words).toarray()
X_test_ng = ngram_vectorizer.transform(test_words).toarray()

In [ ]:
# tfidf
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer(smooth_idf=False);
X_train_tfidf = transformer.fit_transform(X_train_counts);
X_test_tfidf = transformer.transform(X_test_counts);


In [ ]:
# pretrained word2vec
import gensim.downloader as api
wv_pretrain = api.load('word2vec-google-news-300')

In [ ]:
# custom word2vec
import gensim
model_w2v = gensim.models.Word2Vec(sentences=df_train['words'])

In [ ]:
!wget https://github.com/allenai/spv2/raw/master/model/glove.6B.100d.txt.gz
!gzip -d -f glove.6B.100d.txt.gz
!mv glove.6B.100d.txt content

In [ ]:
# pretrained glove
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = 'content/glove.6B.100d.txt'
tmp_file = "content/glove_100d_word2vec.txt"

_ = glove2word2vec(glove_file, tmp_file)

glove_model = KeyedVectors.load_word2vec_format(tmp_file)


In [ ]:
# pretrained bert
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras import Model
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text # just needed tensorflow_text

bert_encoder_dir = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1'
bert_preprocess_dir = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

bert_preprocess_layer = hub.KerasLayer(bert_preprocess_dir)
bert_encode_model = hub.KerasLayer(bert_encoder_dir, trainable=True)

In [ ]:
# tokenized seq for keras
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train['words'])
encoded_train_docs = tokenizer.texts_to_sequences(df_train['words'])
encoded_test_docs = tokenizer.texts_to_sequences(df_test['words'])
max_length = max([len(s) for s in df_train['words']])
X_train = pad_sequences(encoded_train_docs, 
                        maxlen=max_length, 
                        padding='post')
y_train = pd.get_dummies(df_train['category'])
X_test = pad_sequences(encoded_test_docs, 
                        maxlen=max_length, 
                        padding='post')
y_test = pd.get_dummies(df_test['category'])

Examples:

In [ ]:
# plsa using tfidf train data

from sklearn.decomposition import NMF;
from sklearn.preprocessing import normalize;

def get_nmf_topics(model, n_top_words):
    
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = count_vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(num_topics):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-n_top_words - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict)

xtfidf_norm = normalize(X_train_tfidf, norm='l1', axis=1)
num_topics=10
plsa_model = NMF(n_components=num_topics, init='nndsvd');
plsa_model.fit(xtfidf_norm)
get_nmf_topics(plsa_model, 20)

In [ ]:
# word2vec + cnn
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Conv1D, MaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

def get_word2vec_embed_layer(max_length, tokenizer, wv):
    word_index = tokenizer.word_index
    embedding_mat = np.zeros((len(word_index)+1, 100))
    for word, i in word_index.items():
        try:
            vector = wv[word]
            embedding_mat[i] = vector
        except:
            continue
    word2vec_embedding_layer = Embedding(input_dim=embedding_mat.shape[0],
                          output_dim=embedding_mat.shape[1], 
                          weights=[embedding_mat],
                          input_length=max_length, 
                          trainable=False)
    return word2vec_embedding_layer

model_word2vec = Sequential()
model_word2vec.add(get_word2vec_embed_layer(max_length, tokenizer, model_w2v.wv)) # use wv_pretrain instead of model_w2v.wv for pretrain w2v
model_word2vec.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model_word2vec.add(MaxPooling1D(pool_size=2))
model_word2vec.add(Flatten())
model_word2vec.add(Dense(11, activation='softmax'))
print(model_word2vec.summary())
model_word2vec.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_word2vec.fit(X_train, y_train, epochs=10)
model_word2vec.evaluate(X_test, y_test)

In [ ]:
# bert + cnn
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras import Model
import tensorflow as tf
import tensorflow_hub as hub

text_input = Input(shape=(), dtype=tf.string)
bert_inputs = bert_preprocess_layer(text_input)
outputs = bert_encode_model(bert_inputs)
net = outputs['pooled_output']
net = Dropout(0.1)(net)
net = Dense(11, activation='softmax')(net)
bert_model = Model(text_input, net)
bert_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
bert_model.fit(tf.constant(train_words), y_train, epochs=10)


In [ ]:
# pretrained glove

def get_glove_embed_layer(max_length, tokenizer, wv):
    word_index = tokenizer.word_index
    embedding_glove_mat = np.zeros((len(word_index)+1, 100))
    for word, i in word_index.items():
        try:
            vector = glove_model[word]
            embedding_glove_mat[i] = vector
        except:
            continue
    glove_embedding_layer = Embedding(input_dim=embedding_glove_mat.shape[0],
                          output_dim=embedding_glove_mat.shape[1], 
                          weights=[embedding_glove_mat],
                          input_length=max_length, 
                          trainable=False)
    return glove_embedding_layer

model_glove = Sequential()
model_glove.add(get_glove_embed_layer(max_length, tokenizer, model_w2v.wv))
model_glove.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=2))
model_glove.add(Flatten())
model_word2vec.add(Dense(11, activation='softmax'))
print(model_word2vec.summary())
model_word2vec.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_word2vec.fit(X_train, y_train, epochs=10)
model_word2vec.evaluate(X_test, y_test)